# How to create interactive Structure-like plots

by [@ODiogoSilva](https://twitter.com/ODiogoSilva)

<br>
If you have already used Structure or any Structure-like software (fastStructure, MavericK), you may be familiar with the stacked bar plot that is usually generated at the end of the analysis. This plot is a nice visual representation of how individuals are structured within a population, but I always felt that exploration of such a static plot was difficult, particularly with many individuals.

As a result, I used **python** and the awesome **plotly** library to generate nice interactive plots from the output of such programs:

In [82]:
%run ./plot_script.py

If you think this cool but have no idea on how to do it, I've got you covered. Here I'll show a step by step tutorial so that you can start producing your own interactive plots.

# What do you need?

- python3.x: Plotly provides an API for many languages, but here we'll use python3.

#### Python modules

- plotly
- numpy
- colorlover

These are the basic modules required to run this tutorial. If you don't already have them, you can easily install them via `pip`

In [ ]:
pip install plotly numpy colorlover

#### Test data

This tutorial assumes that you have a structure output file (in this case I'll start with a fastStructure `meanQ` output file). You'll also need an individuals file that maps each column to a particular sample name.

If you don't have any testing data, you can use [this meanQ file](https://github.com/ODiogoSilva/shared_dump/raw/master/interactive_structure_plot/HvFullSample_ingroup_v1_MM50_mafRandSNP.4.meanQ) and [this individuals file](https://github.com/ODiogoSilva/shared_dump/raw/master/interactive_structure_plot/indfile.txt) locally, or use the remote files as in the examples below. 

# Let's get started!

Let's start by importing the required python modules

In [83]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import numpy as np
import colorlover as cl

# You don't need this next line. It's just to display plots in the notebook
init_notebook_mode(connected=True)

## Importing a meanQ file

First we need to get the data that will be plotted. If you already have your own in a numpy array, you can skip until the [plotting section](#Let-the-plotting-begin).

If you open a meanQ file from fastStructure, you can see that it's a pretty simple text file with an array of values. We can easily import this data into a numpy array using the `genfromtxt` method.

In [84]:
# You can import the file directly from the web or download it and import as a local file
qvals = np.genfromtxt("https://github.com/ODiogoSilva/shared_dump/raw/master/interactive_structure_plot/HvFullSample_ingroup_v1_MM50_mafRandSNP.5.meanQ")

Now we have the matrix with the information required to produce the plot in the `qvals` variable. Numpy arrays have a `shape` attribute that can inform us of the dimensionality of the matrix. In other words, we can know the number of clusters (K) and individuals in the matrix.

In [85]:
print(qvals.shape)

(101, 5)


This means that our meanQ matrix has 101 individuals and 5 clusters.

However, when creating the plot, it may be more convenient to have a transposed matrix of 5 rows and 101 columns. Fortunatelly, transposing numpy matrices is quite easy.

In [10]:
qvals_transposed = qvals.T

print(qvals_transposed.shape)

(5, 101)


This will be the variable that we'll use to create the plot.

## Importing an individuals files

Now we need to import the individuals file so that we can later map our individuals and/or populations to the plot bars. Once again, the `genfromtxt` method of numpy comes to the rescue! 

We will only need one modification from the previous usage. Since `genfromtxt` assumes that the default data type is `float` (and individual and population names are rarely represented as `floats`), we need to change the data type to, for instance, 20-character unicode.

In [86]:
indarray = np.genfromtxt("https://github.com/ODiogoSilva/shared_dump/raw/master/interactive_structure_plot/indfile.txt",
                         dtype="|U20")

# Print the first two rows
print(indarray[:2,:])

[['Sample1' 'Pop1']
 ['Sample2' 'Pop4']]


As you can see, the individuals file was correctly imported into an array of strings. We can use the `shape` attribute again to confirm that we have 101 individuals in the array.

In [12]:
print(indarray.shape)

(101, 2)


For now, we'll be only interested in mapping the individual sample names, so let's store this information in a 1D array

In [13]:
# Get all rows from the first column
individuals = indarray[:,0]
print(individuals)

['Sample1' 'Sample2' 'Sample3' 'Sample4' 'Sample5' 'Sample6' 'Sample7'
 'Sample8' 'Sample9' 'Sample10' 'Sample11' 'Sample12' 'Sample13' 'Sample14'
 'Sample15' 'Sample16' 'Sample17' 'Sample18' 'Sample19' 'Sample20'
 'Sample21' 'Sample22' 'Sample23' 'Sample24' 'Sample25' 'Sample26'
 'Sample27' 'Sample28' 'Sample29' 'Sample30' 'Sample31' 'Sample32'
 'Sample33' 'Sample34' 'Sample35' 'Sample36' 'Sample37' 'Sample38'
 'Sample39' 'Sample40' 'Sample41' 'Sample42' 'Sample43' 'Sample44'
 'Sample45' 'Sample46' 'Sample47' 'Sample48' 'Sample49' 'Sample50'
 'Sample51' 'Sample52' 'Sample53' 'Sample54' 'Sample55' 'Sample56'
 'Sample57' 'Sample58' 'Sample59' 'Sample60' 'Sample61' 'Sample62'
 'Sample63' 'Sample64' 'Sample65' 'Sample66' 'Sample67' 'Sample68'
 'Sample69' 'Sample70' 'Sample71' 'Sample72' 'Sample73' 'Sample74'
 'Sample75' 'Sample76' 'Sample77' 'Sample78' 'Sample79' 'Sample80'
 'Sample81' 'Sample82' 'Sample83' 'Sample84' 'Sample85' 'Sample86'
 'Sample87' 'Sample88' 'Sample89' 'Sample90' 'Sam

## Let the plotting begin

### Basic plot

Plotly has loads and loads of options to create and customize your plots, so the code for the final plot can be a bit overwhelming at first. So, let's start with just the simplest plot and then we will gradually build it from there.

The following code will be enough to generate a basic (and not very attractive) interactive stacked bar plot from our `meanQ` and individuals data.

In [69]:
# Initialize variable that will store the trace of the Bars for each K 
data = []

# The enumerate() function provides a convenient way of getting the K value in each iteration,
# since it returns the index of the iterable.
for k, ar in enumerate(qvals_transposed):
    
    # Let's create the trace for the current K group
    current_bar = go.Bar(
        # Provide the list of individuals as the values of the x-axis
        x = individuals,
        # Provide the meanQ values for the current K
        y = ar,
        # Here you can name your clusters and use the 'k' variable as a counter
        name = "K {}".format(k)
    )
    
    # Append the current trace to the storage
    data.append(current_bar)
    
# Set the layout of the plot as 'stack' to produce a stacked bar pot
layout = go.Layout(
    barmode='stack',
    margin={"t":0}
)

# Provide data and layout to fig object
fig = go.Figure(data=data, layout=layout)

# Plot the figure object. You can use the 'plot()' method instead of 'iplot()' to generate an html file
iplot(fig)

As you can see, with just a few lines of code, you can create your own structure interactive plot. However, there is still much room for improvement to make it more "structure" like. 

### Customizing axis labels

Let's start with the labels on the axis. We'll remove the ticks from the y-axis labels and customize the individual names a bit. As you can see, this can be easily done by modifying the `Layout()` object.

In [87]:
data = []

for k, ar in enumerate(qvals_transposed):
    
    current_bar = go.Bar(
        x = individuals,
        y = ar,
        name = "K {}".format(k)
    )
    
    data.append(current_bar)
    
layout = go.Layout(
    barmode='stack',
    margin={"t":0},
    # Disable labels form y-axis ticks
    yaxis={
        "showticklabels":False
    },
    # Set the rotation and font of the x-axis ticks
    xaxis={
        "tickangle":-45,
        "tickfont": dict(size=14,
                        color="black")
    }
)


fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Customizing the bars

Next, we'll remove the space between the individual bars and change the color scheme by modifying `Bar()` and `Layout()`.

You can also add information to the labels that pop up when the mouse hovers over the bars. This information is provided via the `text` argument of the `Bar()` object. You must provide a list of the same lenght as the number of individuals. Each entry corresponds to the text for an individual. In the code below we set the text as the assignment probability (in percentage) for each individual sample.

If you want to differentiate between individual bars, you can modify the maker line width in `Bar()`.

In [88]:
data = []

# Create color pallete for 12 colors
# More color pallets available on https://plot.ly/ipython-notebooks/color-scales/
cp = cl.scales["12"]["qual"]["Set3"]

for k, ar in enumerate(qvals_transposed):
    
    current_bar = go.Bar(
        x = individuals,
        y = ar,
        name = "K {}".format(k),
        # Set hover text information.
        text=["Assignment: {}%".format(x * 100) for x in ar],
        # Customize bars
        marker = dict(
            # Use the 'k' counter to fetch a different color for each K value
            color=cp[k],
            line=dict(
                color="grey",
                # Modify the width to values greater than 0 to show lines around the individual bars
                width=0,
            )
        )
    )
    
    data.append(current_bar)
    
layout = go.Layout(
    barmode='stack',
    margin={"t":0},
    bargap="0",
    yaxis={
        "showticklabels":False
    },
    xaxis={
        "tickangle":-45,
        "tickfont": dict(size=14,
                        color="black")
    }
)


fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Set plot border and center legend

We can also set a border around the plot by creating Line shapes and adding them to the layout. First, we create and populated the `shape_list` variable with the line borders information, and then we provide this information to the `Layout()` object.

To change the position of the legend you can tweak the `x` and `y` relative position in the `Layout()` object.

In [89]:
data = []

# Create color pallete for 12 colors
# More color pallets available on https://plot.ly/ipython-notebooks/color-scales/
cp = cl.scales["12"]["qual"]["Set3"]

for k, ar in enumerate(qvals_transposed):
    
    current_bar = go.Bar(
        x = individuals,
        y = ar,
        name = "K {}".format(k),
        text=["Assignment: {}%".format(x * 100) for x in ar],
        marker = dict(
            color=cp[k],
            line=dict(
                color="grey",
                width=0,
            )
        )
    )
    
    data.append(current_bar)

number_individuals = len(individuals)

# Store each line surrounding the plot in a list 
# Note that some adjustments were necessary to prevent the lateral lines from cutting the first and last bars
shape_list = [
    # Bottom line
    {"type": "line", "x0": - .5, "y0": 0,
     "x1": number_individuals - .5, "y1": 0,
     "line": {"width": 2}},
    # Left line
    {"type": "line", "x0": - .5, "y0": 0, "x1": -.5, "y1": 1,
     "line": {"width": 3}},
    # Top line
    {"type": "line", "x0": - .5, "y0": 1,
     "x1": number_individuals - .5, "y1": 1,
     "line": {"width": 3}},
    # Right line
    {"type": "line", "x0": number_individuals -.5, "y0": 0,
     "x1": number_individuals - .5, "y1": 1,
     "line": {"width": 3}}
]

    
layout = go.Layout(
    barmode='stack',
    margin={"t":0},
    bargap="0",
    # Add the shapes to the layout.
    shapes=shape_list,
    # Set the legend position to the middle left
    legend={"x":1, "y":0.5},
    yaxis={
        "showticklabels":False
    },
    xaxis={
        "tickangle":-45,
        "tickfont": dict(size=14,
                        color="black")
    }
)


fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Including population boundaries

Now it gets a little bit tricky. Until now we have only generated a plot with individual sampes, but you may want to sort and separate your individuals by population. 

You may have noticed that the test individuals file has a second column with the information of the populations. If you print only the second column of the array you can see the population assigned to each individual sample.

In [99]:
print(indarray[:,1])

['Pop1' 'Pop4' 'Pop1' 'Pop2' 'Pop1' 'Pop2' 'Pop2' 'Pop1' 'Pop1' 'Pop2'
 'Pop1' 'Pop3' 'Pop1' 'Pop2' 'Pop1' 'Pop4' 'Pop1' 'Pop1' 'Pop1' 'Pop2'
 'Pop1' 'Pop1' 'Pop2' 'Pop1' 'Pop4' 'Pop2' 'Pop4' 'Pop2' 'Pop4' 'Pop1'
 'Pop1' 'Pop1' 'Pop1' 'Pop1' 'Pop4' 'Pop2' 'Pop4' 'Pop1' 'Pop2' 'Pop1'
 'Pop1' 'Pop1' 'Pop1' 'Pop1' 'Pop2' 'Pop1' 'Pop1' 'Pop3' 'Pop1' 'Pop1'
 'Pop2' 'Pop4' 'Pop4' 'Pop2' 'Pop4' 'Pop1' 'Pop2' 'Pop1' 'Pop4' 'Pop1'
 'Pop5' 'Pop4' 'Pop4' 'Pop4' 'Pop2' 'Pop1' 'Pop4' 'Pop1' 'Pop4' 'Pop2'
 'Pop4' 'Pop2' 'Pop5' 'Pop4' 'Pop1' 'Pop1' 'Pop1' 'Pop1' 'Pop1' 'Pop1'
 'Pop1' 'Pop4' 'Pop1' 'Pop2' 'Pop4' 'Pop4' 'Pop1' 'Pop3' 'Pop1' 'Pop1'
 'Pop1' 'Pop1' 'Pop2' 'Pop4' 'Pop2' 'Pop4' 'Pop1' 'Pop4' 'Pop1' 'Pop4'
 'Pop1']


To further complicate things, individuals are not even sorted by population. So, if we want to correctly display the plot with individuals sorted by populations, we need to sort both the meanQ and individuals arrays.

First, let's put the array with the populations in a new variable that will be used as an index.

In [100]:
index = indarray[:, 1]

Now let's concatenate this index at the beggining of the meanQ array to that we can then sort by that column.

In [101]:
index_array = np.c_[index, qvals]
print(index_array[0:2,:])

[['Pop1' '7.1e-05' '7.1e-05' '0.999717' '7.1e-05' '7.1e-05']
 ['Pop4' '8.6e-05' '8.5e-05' '8.6e-05' '0.999658' '8.5e-05']]


As you can see, the first column of `index_array` contains the population information. We can now sort the meanQ array by this column using some numpy `argsort()` magic.

In [102]:
sorted_qvals = index_array[index_array[:, 0].argsort()]
print(sorted_qvals[0:2,:])

[['Pop1' '7.1e-05' '7.1e-05' '0.999717' '7.1e-05' '7.1e-05']
 ['Pop1' '9e-05' '9e-05' '0.99964' '9e-05' '9e-05']]


Now that the array is sorted by population, we can remove the first column and defined the `sorted_qvals` array with the sorted version of `qvals`. We also need to convert the array to `float` again.

In [103]:
sorted_qvals = sorted_qvals[:,1:].astype(np.float64)
print(sorted_qvals[0:2,:])

sorted_qvals_transposed = sorted_qvals.T

[[  7.10000000e-05   7.10000000e-05   9.99717000e-01   7.10000000e-05
    7.10000000e-05]
 [  9.00000000e-05   9.00000000e-05   9.99640000e-01   9.00000000e-05
    9.00000000e-05]]


Now we only need to sort the `indarray` variable, which is much simpler.

In [104]:
sorted_indarray = indarray[indarray[:,1].argsort()]
sorted_individuals = sorted_indarray[:,0]

So, now that we finally have sorted version of the meanQ and individuals arrays we can proceed to the plot.

Let's start by creating boundaries for each population. For this, we will need to calculate the positions of each boundary along the x-axis. Here's one way of doing that.

In [105]:
from collections import OrderedDict, Counter

# Get sorted unique list of populations
# E.g.: ['Pop1', 'Pop2', 'Pop3', 'Pop4', 'Pop5']
pops = sorted(list(OrderedDict.fromkeys(indarray[:, 1])))

# Get cumulative sums of individuals in each population
# E.g.: [ 50  71  74  99 101]
# Meaning that the first population has 72 individuals, the second has 9, etc.
pop_sums = np.cumsum([np.count_nonzero(indarray == x)
                                      for x in pops])

# Get the number of individuals in each population
pop_counts = Counter(indarray[:, 1])

# Create list with the ranges for each population
pop_ranges = []
# Create list with the xpos of each population label
pop_xpos = []
for counter, pop in enumerate(pops):
    pop_ranges.append((pop_sums[counter] - pop_counts[pop], pop_sums[counter]))
    pop_xpos.append(pop_sums[counter] - pop_counts[pop] / 2)
    
print(pop_ranges)

[(0, 50), (50, 71), (71, 74), (74, 99), (99, 101)]


And now we add these boundaries to the `shape_list` variable, already using the sorted arrays.

In [108]:
data = []

# Create color pallete for 12 colors
# More color pallets available on https://plot.ly/ipython-notebooks/color-scales/
cp = cl.scales["12"]["qual"]["Set3"]

for k, ar in enumerate(sorted_qvals_transposed):
    
    current_bar = go.Bar(
        x = sorted_individuals,
        y = ar,
        name = "K {}".format(k),
        text=["Assignment: {}%".format(x * 100) for x in ar],
        marker = dict(
            color=cp[k],
            line=dict(
                color="grey",
                width=0,
            )
        )
    )
    
    data.append(current_bar)

number_individuals = len(individuals)

shape_list = [
    {"type": "line", "x0": - .5, "y0": 0,
     "x1": number_individuals - .5, "y1": 0,
     "line": {"width": 2}},
    {"type": "line", "x0": - .5, "y0": 0, "x1": -.5, "y1": 1,
     "line": {"width": 3}},
    {"type": "line", "x0": - .5, "y0": 1,
     "x1": number_individuals - .5, "y1": 1,
     "line": {"width": 3}},
    {"type": "line", "x0": number_individuals -.5, "y0": 0,
     "x1": number_individuals - .5, "y1": 1,
     "line": {"width": 3}}
]

# Adding the population boundaries to the shape_list
# Simplify the population ranges into a list with the x-axis position of the boundaries
pop_lines = list(
    OrderedDict.fromkeys([x for y in pop_ranges for x in y]))[1:-1]
# Add vertical lines at those positions
for xpos in pop_lines:
    shape_list.append(
         {"type": "line",
          "x0": xpos - .5,
          "y0": 0,
          "x1": xpos - .5,
          "y1": 1,
          "line": {"width":3}}
    )
    
layout = go.Layout(
    barmode='stack',
    margin={"t":0},
    bargap="0",
    shapes=shape_list,
    legend={"x":1, "y":0.5},
    yaxis={
        "showticklabels":False
    },
    xaxis={
        "tickangle":-45,
        "tickfont": dict(size=14,
                        color="black")
    }
)


fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Using population labels

And finally, we can remove the individual sample names and provide only the population labels in the x-axis.

In [109]:
data = []

# Create color pallete for 12 colors
# More color pallets available on https://plot.ly/ipython-notebooks/color-scales/
cp = cl.scales["12"]["qual"]["Set3"]

for k, ar in enumerate(sorted_qvals_transposed):
    
    current_bar = go.Bar(
        x = sorted_individuals,
        y = ar,
        name = "K {}".format(k),
        text=["Assignment: {}%".format(x * 100) for x in ar],
        marker = dict(
            color=cp[k],
            line=dict(
                color="grey",
                width=1,
            )
        )
    )
    
    data.append(current_bar)

number_individuals = len(individuals)

shape_list = [
    {"type": "line", "x0": - .5, "y0": 0,
     "x1": number_individuals - .5, "y1": 0,
     "line": {"width": 2}},
    {"type": "line", "x0": - .5, "y0": 0, "x1": -.5, "y1": 1,
     "line": {"width": 3}},
    {"type": "line", "x0": - .5, "y0": 1,
     "x1": number_individuals - .5, "y1": 1,
     "line": {"width": 3}},
    {"type": "line", "x0": number_individuals -.5, "y0": 0,
     "x1": number_individuals - .5, "y1": 1,
     "line": {"width": 3}}
]

pop_lines = list(
    OrderedDict.fromkeys([x for y in pop_ranges for x in y]))[1:-1]
for xpos in pop_lines:
    shape_list.append(
         {"type": "line",
          "x0": xpos - .5,
          "y0": 0,
          "x1": xpos - .5,
          "y1": 1,
          "line": {"width":3}}
    )
    
layout = go.Layout(
    # Add title to plot
    title="Interactive structure plot",
    barmode="stack",
    # Adjust top margin for title
    margin={"t":40},
    bargap="0",
    shapes=shape_list,
    legend={"x":1, "y":0.5},
    yaxis={
        "showticklabels":False
    },
    xaxis={
        # Disable the xticks with individual names
        "ticks": "",
        "showticklabels": True,
        # Provide your custom tick labels
        "ticktext": pops,
        # And their positions
        "tickvals": pop_xpos,
        "tickangle":-45,
        "tickfont": dict(size=22,
                        color="black")
    }
)


fig = go.Figure(data=data, layout=layout)

iplot(fig)

And there you have it, a nice and interactive structure plot that can be vastly customized to suit your needs. :D